In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import ExtraTreeRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure your code runs with RUN ALL. It should load the data that you're given, do all the processing, and spit out results. Comment out or remove anything that you've cleaned up and don't need - e.g. if you scaled a value manually, then moved that into a pipeline, don't leave the original scaling code active when the file is run.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a markdown cell at the bottom, and put in a few notes addressing the following:
    <ul>
        <li> Accuracy of your models with/without feature selection. Include both train/test for each. Please use R2 and RMSE. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Hyperparameter Changes / Grid Search Improvements. What did you try, and why. Similar explaination to above, short. 
        <li> Overall this section should be roughly as long as this intro block - just outline what the results were, what you did to improve, and the results after. 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

In [18]:
#Load Data
df = pd.read_csv("data/Euro_Salary.csv")
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Timestamp,1253,1248,24/11/2020 13:55:19,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,119,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1237,48,10,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of experience in Germany,1221,53,2,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seniority level,1241,24,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,256,Java,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other technologies/programming languages you use often,1096,562,Javascript / Typescript,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df.sample(10)

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
876,27/11/2020 21:29:14,33.0,Male,Frankfurt,Software Engineer,8,5,Senior,C#,"Javascript / Typescript, .NET, SQL",67000.0,6000,28,Full-time employee,Unlimited contract,English,11-50,Product
829,27/11/2020 12:20:51,NaN,Male,Berlin,data engineer,11,1,Middle,python,"Python, SQL, AWS, Google Cloud",68000.0,0,28,Full-time employee,Unlimited contract,English,1000+,Product
970,30/11/2020 15:27:53,29.0,Male,Berlin,Backend Developer,9,6,Senior,Java,"Kotlin, Java / Scala, SQL, AWS",75000.0,NaN,25,Full-time employee,Unlimited contract,English,51-100,Startup
688,25/11/2020 19:54:39,28.0,Male,Karlsruhe,Backend Developer,7,3,Senior,Java,"Kotlin, Javascript / Typescript, AWS, Kubernet...",66000.0,3000,28,Full-time employee,Unlimited contract,English,1000+,Product
268,24/11/2020 14:40:25,32.0,Female,Stuttgart,Stuttgart,2,2,Junior,Java,NaN,52000.0,4000,30,Full-time employee,Unlimited contract,English,1000+,Product
940,30/11/2020 12:06:18,28.0,Male,Berlin,ML Engineer,1,1,Junior,Python,"C/C++, AWS, Google Cloud, Docker",43000.0,43000,14,Full-time employee,Unlimited contract,English,up to 10,Startup
1021,01/12/2020 09:50:38,30.0,Female,Munich,Frontend Developer,9,3,Senior,"JavaScript, TypeScript",PHP,60000.0,4000,28,Full-time employee,Unlimited contract,English,1000+,Product
1113,07/12/2020 22:47:21,32.0,Male,Munich,QA Engineer,12,4,Senior,NaN,NaN,95000.0,8000,25,Full-time employee,Unlimited contract,Russian,1000+,Product
861,27/11/2020 17:18:20,36.0,Male,Berlin,Backend Developer,5,2,Senior,Java,"Java / Scala, Kubernetes, Docker",70000.0,0,28,Full-time employee,Unlimited contract,English,101-1000,Startup
1173,18/12/2020 11:29:23,30.0,Male,Berlin,Backend Developer,9,5,Senior,Scala,"Java / Scala, AWS, Kubernetes",85000.0,90000,24,Full-time employee,Unlimited contract,English,1000+,Product


In [20]:
#df.info()
df["Other technologies/programming languages you use often"].isna().value_counts()

False    1096
True      157
Name: Other technologies/programming languages you use often, dtype: int64

In [21]:
df.head()

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
0,24/11/2020 11:14:15,26.0,Male,Munich,Software Engineer,5,3,Senior,TypeScript,"Kotlin, Javascript / Typescript",80000.0,5000,30,Full-time employee,Unlimited contract,English,51-100,Product
1,24/11/2020 11:14:16,26.0,Male,Berlin,Backend Developer,7,4,Senior,Ruby,NaN,80000.0,NaN,28,Full-time employee,Unlimited contract,English,101-1000,Product
2,24/11/2020 11:14:21,29.0,Male,Berlin,Software Engineer,12,6,Lead,Javascript / Typescript,"Javascript / Typescript, Docker",120000.0,120000,30,Self-employed (freelancer),Temporary contract,English,101-1000,Product
3,24/11/2020 11:15:24,28.0,Male,Berlin,Frontend Developer,4,1,Junior,Javascript,NaN,54000.0,NaN,24,Full-time employee,Unlimited contract,English,51-100,Startup
4,24/11/2020 11:15:46,37.0,Male,Berlin,Backend Developer,17,6,Senior,C# .NET,".NET, SQL, AWS, Docker",62000.0,NaN,29,Full-time employee,Unlimited contract,English,101-1000,Product


#### Create the target column of Total compensation by combining the yearly salary and yearly bonus columns, then dropping them as well as a metadata column

In [22]:
df["Yearly bonus + stocks in EUR"] = pd.to_numeric(df["Yearly bonus + stocks in EUR"], downcast="float", errors="coerce")
df["Yearly bonus + stocks in EUR"].fillna(0, inplace=True)
df['Total Compensation'] = df['Yearly brutto salary (without bonus and stocks) in EUR'] + df['Yearly bonus + stocks in EUR']

df.drop(columns={"Timestamp","Yearly brutto salary (without bonus and stocks) in EUR", "Yearly bonus + stocks in EUR"}, inplace=True)
df['Position '].rename('Position')

0                            Software Engineer
1                            Backend Developer
2                            Software Engineer
3                           Frontend Developer
4                            Backend Developer
                         ...                  
1248                         Backend Developer
1249    Researcher/ Consumer Insights Analyst 
1250                     IT Operations Manager
1251                        Frontend Developer
1252                                    DevOps
Name: Position, Length: 1253, dtype: object

In [23]:
#function to replace the x lowest occurring values in a column with a designated value
def replace_low_freq(d, col, threshold=10, replacement='other'):
    value_counts = d[col].value_counts() # Specific column 
    to_remove = value_counts[value_counts <= threshold].index
    tmp = d[col].replace(to_replace=to_remove, value=replacement)
    return tmp

#### Removing the excessive amounts of long responses in these two columns

In [24]:
#removing the excessive amounts of long responses in these two columns
df["Your main technology / programming language"] = replace_low_freq(df, "Your main technology / programming language", 10, "Other")
df['Other technologies/programming languages you use often'] = replace_low_freq(df, "Your main technology / programming language", 5, "Other")

#### Filling in NaNs with selected data. For categorical columns, they are labeled as unknown. For numerical columns, they are usually filled with the median column value

In [25]:
df['Age'].fillna(df['Age'].median(), inplace=True)

df['Gender'].fillna('Other',inplace=True)
df['Position '].fillna('Other',inplace=True)
df['Seniority level'].fillna('Other',inplace=True)
df['Company type'].fillna('Other',inplace=True)

df['Employment status'].fillna('Unknown',inplace=True)
df['Сontract duration'].fillna('Unknown',inplace=True)
df['Main language at work'].fillna('Unknown',inplace=True)
df['Company size'].fillna('Unknown',inplace=True)
df['Company type'].fillna('Unknown',inplace=True)

df['Years of experience in Germany'] = pd.to_numeric(df['Years of experience in Germany'], downcast="float", errors="coerce")
df['Years of experience in Germany'].fillna(df['Years of experience in Germany'].median(),inplace=True)

df['Number of vacation days'] = pd.to_numeric(df['Number of vacation days'], downcast="float", errors="coerce")
df['Number of vacation days'].fillna(df['Number of vacation days'].median(),inplace=True)

df['Total years of experience'] = pd.to_numeric(df['Total years of experience'], downcast="float", errors="coerce")
df['Total years of experience'].fillna(df['Years of experience in Germany'],inplace=True)

df["Your main technology / programming language"].fillna('Unknown',inplace=True)
df['Other technologies/programming languages you use often'].fillna('Unknown',inplace=True)

#### Creation of the full feature pipeline using a Column Transformer

In [26]:
categorical_features = ['Gender', 'City', 'Position ','Seniority level','Your main technology / programming language',
                        'Other technologies/programming languages you use often','Employment status','Сontract duration',
                        'Main language at work','Company size', 'Company type']
categorical_transformer = OneHotEncoder(handle_unknown = 'ignore')

numeric_features = ["Age", 'Total years of experience','Years of experience in Germany','Number of vacation days']
numeric_transformer = Pipeline( steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", MinMaxScaler())
    ])

preprocessor = ColumnTransformer( transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

rf_para = {"criterion":['squared_error','friedman_mse', 'absolute_error', 'poisson'],
            'max_depth':[1,2,3,4,5],
            'ccp_alpha':[0.020,0.0205,0.021],
            'splitter':['random','best'],
            'max_features':['auto', 'sqrt', 'log2']
            }

#Creating the models, with a Grid Search used on the Extra Tree Regressor
pipeline_steps_grid = [('pre', preprocessor),('search', GridSearchCV(estimator=ExtraTreeRegressor(),n_jobs=-1,param_grid=rf_para))]
grid = Pipeline(pipeline_steps_grid)

#### Train Test Split

In [27]:
y = (df['Total Compensation'])
x = df.drop(columns={'Total Compensation'})

xTrain,xTest,yTrain,yTest = train_test_split(x,y,random_state=0)

#### Fitting and output of RMSE and R2 values of each model

In [28]:
grid.fit(xTrain,yTrain)

yPredG = grid.predict(xTest)

print(grid.named_steps['search'].best_estimator_)
print("RMSE:",mean_squared_error(yTest,yPredG,squared=False),'\n',"R2 Train:",grid.score(xTrain,yTrain),'\n',"R2 Test:",grid.score(xTest,yTest))

c:\Users\atrsy\anaconda3\lib\site-packages\sklearn\tree\_classes.py:306: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


ExtraTreeRegressor(ccp_alpha=0.02, criterion='absolute_error', max_depth=3,
                   max_features='auto')
RMSE: 48366.9487981678 
 R2 Train: 0.001836370027819223 
 R2 Test: 0.07487223667362775


#### Feature selection using Variance Threshold

In [29]:
varTh = VarianceThreshold(.2)
xDummied = pd.get_dummies(x,drop_first=True)
fsX = varTh.fit_transform(xDummied,y)
mask = varTh.get_support()
newFeatures = xDummied.columns[mask]
print(newFeatures)

Index(['Age', 'Total years of experience', 'Years of experience in Germany',
       'Number of vacation days', 'City_Berlin', 'Position _Software Engineer',
       'Seniority level_Middle', 'Seniority level_Senior',
       'Your main technology / programming language_Other',
       'Other technologies/programming languages you use often_Other',
       'Company size_101-1000', 'Company type_Product'],
      dtype='object')


#### Feature Selection with Select K Best

In [30]:
kb = SelectKBest(mutual_info_regression, k=7)
kbX = kb.fit_transform(xDummied,y)
print("K-Best:", kbX.shape)

#Print the features
mask = kb.get_support()
new_features = xDummied.columns[mask]
print("K-Best Features:", new_features)

K-Best: (1253, 7)
K-Best Features: Index(['Age', 'Total years of experience', 'Years of experience in Germany',
       'Number of vacation days', 'Gender_Male', 'Seniority level_Middle',
       'Seniority level_Senior'],
      dtype='object')


#### Feature Selected Pipeline

In [31]:
#creation of the feature selection pipeline.
numericTransformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", MinMaxScaler())
    ])

##Creating the models, with a Grid Search used on the Extra Tree Regressor
pipelineStepsGrid = [('num', numericTransformer),('search', GridSearchCV(estimator=ExtraTreeRegressor(),n_jobs=-1,param_grid=rf_para))]
fsGrid = Pipeline(pipelineStepsGrid)

In [32]:
#Fitting and output of RMSE and R2 values of each model
fsX = pd.DataFrame(fsX,columns=newFeatures)
xTrainFS,xTestFS,yTrainFS,yTestFS = train_test_split(fsX, y,random_state=42)

fsGrid.fit(xTrainFS,yTrainFS)
yPredFSG = fsGrid.predict(xTestFS)

print("Variance Threshold",fsGrid.named_steps['search'].best_estimator_)
print('RMSE:',mean_squared_error(yTestFS,yPredFSG,squared=False),'\n',"R2 Train:",fsGrid.score(xTrainFS,yTrainFS),'\n',"R2 Test:",fsGrid.score(xTestFS,yTestFS),'\n')

kbX = pd.DataFrame(kbX,columns=new_features)
xTrainFS,xTestFS,yTrainFS,yTestFS = train_test_split(kbX, y,random_state=42)

fsGrid.fit(xTrainFS,yTrainFS)
yPredFSG = fsGrid.predict(xTestFS)

print("Select K Best",fsGrid.named_steps['search'].best_estimator_)
print('RMSE:',mean_squared_error(yTestFS,yPredFSG,squared=False),'\n',"R2 Train:",fsGrid.score(xTrainFS,yTrainFS),'\n',"R2 Test:",fsGrid.score(xTestFS,yTestFS))

c:\Users\atrsy\anaconda3\lib\site-packages\sklearn\tree\_classes.py:306: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


Variance Threshold ExtraTreeRegressor(ccp_alpha=0.021, criterion='absolute_error', max_depth=4,
                   max_features='auto')
RMSE: 70687.5179150885 
 R2 Train: 0.001836245177148088 
 R2 Test: -0.004864822410890035 

Select K Best ExtraTreeRegressor(ccp_alpha=0.021, criterion='absolute_error', max_depth=5,
                   max_features='sqrt', splitter='best')
RMSE: 70150.86446037111 
 R2 Train: 0.001835885854396646 
 R2 Test: 0.010334951780437085


# Answers and Explainations
(Expand/modify as needed)

### Results
<ul>
    <li>The best model seemed to be the Extra Tree Regressor, having an RMSE of 50971.06, a Training R2 of -0.00118, and a Testing R2 of -0.0274
    <li>After feature selection, best RMSE was 67007.472, Training R2 was 0.00184 and Testing R2 was 0.0970
</ul>

### Feature Selection Activities
<ul>
    <li>Variance Threshold, removed 4 features
    <li>Select K Best, kept 7 features
</ul>  

### Hyperparameter Changes
<ul>
    <li>Criterion - squared_error, friedman_mse, absolute_error, poisson
    <li>Max_depth - 1, 2, 3, 4, 5
    <li>Ccp_alpha - 0.020, 0.0205, 0.021
    <li>Splitter - random, best
    <li>Max_features - auto, sqrt, log2
</ul>